In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Melanoma"
cohort = "GSE200904"

# Input paths
in_trait_dir = "../../input/GEO/Melanoma"
in_cohort_dir = "../../input/GEO/Melanoma/GSE200904"

# Output paths
out_data_file = "../../output/preprocess/Melanoma/GSE200904.csv"
out_gene_data_file = "../../output/preprocess/Melanoma/gene_data/GSE200904.csv"
out_clinical_data_file = "../../output/preprocess/Melanoma/clinical_data/GSE200904.csv"
json_path = "../../output/preprocess/Melanoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"NanoString nCounter NF1-mutant melanoma gene expression"
!Series_summary	"Purpose: Digital spatial profiling was used to elucidate the interplay between the tumor cells and the surrounding microenvironment FFPE tissues."
!Series_summary	"Methods: Two tissue microarrays (TMA) were generated from each FFPE block using the ISNET Galileo CK4500 computer-driven tissue microarray. Images of H&E-stained cores were demarcated by a pathologist and were utilized alongside NanoString immunofluorescent staining for morphology markers S100, CD45, CD3, and DAPI to draw region of interest."
!Series_overall_design	"Multiplex gene expression analysis covers 73 endogenous genes from 24 different immune cell types and 48 gene-derived signatures measuring biological variables crucial to the tumor-immune interaction."
Sample Characteristics Dictionary:
{0: ['scan_id: 21-002-A_4', 'scan_id: 21-002-A_5', 'scan_id: 21-002-B_1', 'scan_id: 21-002-B_2'], 1: ['roi_id: 1', 'r

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# From the background information, this dataset contains gene expression data
# "Multiplex gene expression analysis covers 73 endogenous genes..."
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# Looking at the sample characteristics dictionary:

# There is no explicit melanoma data in sample characteristics
# Since this is a study specifically about NF1-mutant melanoma (from title),
# all samples are likely melanoma samples with no control group
trait_row = None

# No age information available in the sample characteristics
age_row = None

# No gender information available in the sample characteristics
gender_row = None

# Define conversion functions (even though we don't have data for these variables)
def convert_trait(value):
    """Convert trait data to binary (1 for melanoma, 0 for control)."""
    if not value or ":" not in value:
        return None
    val = value.split(":", 1)[1].strip().lower()
    if "melanoma" in val:
        return 1
    elif "control" in val or "normal" in val or "healthy" in val:
        return 0
    return None

def convert_age(value):
    """Convert age data to continuous numeric values."""
    if not value or ":" not in value:
        return None
    val = value.split(":", 1)[1].strip()
    try:
        return float(val)
    except ValueError:
        return None

def convert_gender(value):
    """Convert gender data to binary (0 for female, 1 for male)."""
    if not value or ":" not in value:
        return None
    val = value.split(":", 1)[1].strip().lower()
    if val in ["female", "f"]:
        return 0
    elif val in ["male", "m"]:
        return 1
    return None

# 3. Save Metadata
# Initial filtering on dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None, we'll skip the clinical feature extraction step


False

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.

Gene data extraction result:
Number of rows: 84
First 20 gene/probe identifiers:
Index(['AKT1', 'ARG1', 'B2M', 'BATF3', 'BCL2', 'CCL5', 'CCND1', 'CD27',
       'CD274', 'CD276', 'CD3E', 'CD4', 'CD40', 'CD40LG', 'CD44', 'CD47',
       'CD68', 'CD74', 'CD86', 'CD8A'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing the gene identifiers in the gene expression data
# These appear to be standard human gene symbols (like AKT1, CD8A, B2M, etc.)
# rather than probe IDs or other identifiers that would need mapping

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
print(f"Top 10 gene indices before normalization: {gene_data.index[:10].tolist()}")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Top 10 gene indices after normalization: {normalized_gene_data.index[:10].tolist()}")
print(f"Shape of normalized gene data: {normalized_gene_data.shape}")

# Create directory for gene data file if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (no appropriate trait data available),
# we can't properly link clinical and gene data for trait association analysis
print("No appropriate trait information available for case-control comparison.")

# Create a minimal DataFrame with expected structure for validation
minimal_df = pd.DataFrame({trait: [float('nan')], 'dummy_gene': [0]})
# Set is_biased to True since we lack appropriate trait data for comparison
is_biased = True

# Validate and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=False,  # We confirmed trait data is not available
    is_biased=is_biased,  # Dataset considered biased due to lack of controls
    df=minimal_df,  # Minimal DataFrame with expected structure
    note="Dataset contains gene expression data from NF1-mutant melanoma samples, but lacks control samples for comparison."
)

print("Dataset lacks appropriate trait information (no control samples). Final linked data not created.")

Top 10 gene indices before normalization: ['AKT1', 'ARG1', 'B2M', 'BATF3', 'BCL2', 'CCL5', 'CCND1', 'CD27', 'CD274', 'CD276']
Top 10 gene indices after normalization: ['AKT1', 'ARG1', 'B2M', 'BATF3', 'BCL2', 'CCL5', 'CCND1', 'CD27', 'CD274', 'CD276']
Shape of normalized gene data: (76, 341)
Saved normalized gene data to ../../output/preprocess/Melanoma/gene_data/GSE200904.csv
No appropriate trait information available for case-control comparison.
Abnormality detected in the cohort: GSE200904. Preprocessing failed.
Dataset lacks appropriate trait information (no control samples). Final linked data not created.
